In [ ]:
import pandas as pd  
pd.options.plotting.backend = 'plotly'
pd.set_option('display.max_columns', None)

import numpy as np
from tqdm.notebook import tqdm

In [ ]:
%%time

df = pd.read_parquet('../data/bbva.parquet')

df.head()

In [ ]:
productos = sorted([c for c in df.columns if 'turnover_rate' in c])

cantidad = [e.replace('rate', 'amount') for e in productos]

seleccion = [e for tupla in zip(cantidad, productos) for e in tupla]

columnas = ['x_cliente', 'cutoff_date'] + seleccion

In [ ]:
data=df[columnas]

In [ ]:
data = data.rename(columns={'x_cliente': 'id_cliente'})

In [ ]:
data.head()

In [ ]:
medias = {'extracash_turnover_amount': [5986.736567, 14522.404827],
          'extracash_turnover_rate': [53.319243, 21.855523],
          'pluscash_turnover_amount': [3889.267133, 8101.797840],
          'pluscash_turnover_rate': [61.226405, 25.169930],
          'plussurrogate_turnover_amount': [12658.592996, 15504.792929],
          'plussurrogate_turnover_rate': [15.472557, 4.408297],
          'surrogate_turnover_amount': [7120.014856, 11636.060459],
          'surrogate_turnover_rate': [38.422243, 28.806665]
         }

In [ ]:
fechas = pd.date_range(start='2021-01-01',end='2022-12-31')

len(fechas)

In [ ]:
clientes = data.id_cliente.unique()[:len(fechas)]

len(clientes)

In [ ]:
%%time

final_data = pd.DataFrame()

for i in tqdm(range(20000)):  # operaciones
    
    
    tmp_data = pd.DataFrame()
       
    tmp_data['client_id'] = clientes
    
    tmp_data['date']=fechas
    
    for k,v in medias.items():
        
        tmp_data[k]=np.random.normal(v[0], v[1], size=len(fechas))
        tmp_data[k]=tmp_data[k].apply(lambda x: abs(x))
    
    final_data=pd.concat([final_data, tmp_data])
    

In [ ]:
rates = [c for c in final_data.columns if '_rate' in c]

for r in rates:
    final_data[r]=final_data[r].apply(lambda x: x if x<=100 else x-100)

In [ ]:
for r in rates:
    final_data[r]=final_data[r].apply(lambda x: x+20).round()

In [ ]:
for r in rates:
    final_data[r]=final_data[r].apply(lambda x: 100 if x>100 else x)

In [ ]:
final_data.describe()

In [ ]:
final_data.extracash_turnover_rate.min()

In [ ]:
final_data.to_parquet('../data/final_data.parquet')